In [1]:
import os
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from joblib import Parallel, delayed
from glob import glob
import subprocess
import shutil

# defining the paths for atlas

In [2]:
root_path = "/media/say26747/EC2426FA2426C782/Continual_learning_data/ATLAS/ATLAS_R2.0/ATLAS_2/Training"
modality_paths = sorted(
    glob(os.path.join(root_path, "R*", "sub*", "*", "*", "*_T1w.nii.gz"))
)
seg_paths = sorted(
    glob(os.path.join(root_path, "R*", "sub*", "*", "*", "*_mask.nii.gz"))
)
print(len(modality_paths), len(seg_paths))
print(modality_paths[0], seg_paths[0])

655 655
/media/say26747/EC2426FA2426C782/Continual_learning_data/ATLAS/ATLAS_R2.0/ATLAS_2/Training/R001/sub-r001s001/ses-1/anat/sub-r001s001_ses-1_space-MNI152NLin2009aSym_T1w.nii.gz /media/say26747/EC2426FA2426C782/Continual_learning_data/ATLAS/ATLAS_R2.0/ATLAS_2/Training/R001/sub-r001s001/ses-1/anat/sub-r001s001_ses-1_space-MNI152NLin2009aSym_label-L_desc-T1lesion_mask.nii.gz


In [3]:
save_path = "/media/say26747/EC2426FA2426C782/Continual_learning_data/ATLAS/ATLAS_R2_skull_stripped"
os.makedirs(save_path, exist_ok=True)
robex_path = "/home/say26747/Downloads/ROBEXv12.linux64/ROBEX"

In [4]:
def worker(modality_path, seg_path):
    path_parts_modality = modality_path.split(os.sep)
    # Find and append the part that starts with 'sub-'
    patient_name = None
    for part in path_parts_modality:
        if part.startswith("sub-"):
            patient_name = part
            break

    os.makedirs(os.path.join(save_path, patient_name), exist_ok=True)

    command = f'./ROBEX "{modality_path}" "{save_path}/{patient_name}/{patient_name}_T1w.nii.gz" "{save_path}/{patient_name}/{patient_name}_MASK.nii.gz" 12345'

    try:
        subprocess.run(command, shell=True, check=True)
    except subprocess.CalledProcessError as e:
        print(f"Failed to process {modality_path}: {e}")

    seg_dest_path = os.path.join(save_path, patient_name, f"{patient_name}_Seg.nii.gz")
    try:
        shutil.copy2(seg_path, seg_dest_path)
    except Exception as e:
        print(f"Error: {e}")
    return

In [5]:
os.chdir(robex_path)
results = Parallel(n_jobs=-1)(
    delayed(worker)(modality_path, seg_path)
    for modality_path, seg_path in tqdm(
        zip(modality_paths, seg_paths), total=len(modality_paths)
    )
)

  0%|          | 0/655 [00:00<?, ?it/s]

Step 1  of 9: reading in images...Step 1  of 9: reading in images...Step 1  of 9: reading in images...Step 1  of 9: reading in images...Step 1  of 9: reading in images...Step 1  of 9: reading in images...Step 1  of 9: reading in images...Step 1  of 9: reading in images...Step 1  of 9: reading in images...Step 1  of 9: reading in images...Step 1  of 9: reading in images...Step 1  of 9: reading in images...Done! It took roughly 1 seconds
Step 2 of 9: registration...
  2a) Similarity transform...
Done! It took roughly 1 seconds
Step 2 of 9: registration...
  2a) Similarity transform...
Done! It took roughly 1 seconds
Step 2 of 9: registration...
  2a) Similarity transform...
Done! It took roughly 1 seconds
Step 2 of 9: registration...
  2a) Similarity transform...
Done! It took roughly 1 seconds
Step 2 of 9: registration...
  2a) Similarity transform...
Done! It took roughly 1 seconds
Step 2 of 9: registration...
  2a) Similarity transform...
Done! It took roughly 1 seconds
Step 2 of 9: r